# Why Parquet?

> **Level:** Beginner  
> **Spec:** [Motivation](https://parquet.apache.org/docs/overview/motivation/)  
> **PyArrow docs:** [Reading and Writing the Apache Parquet Format](https://arrow.apache.org/docs/python/parquet.html)

**What you will learn:**

1. Why columnar storage outperforms row-oriented formats for analytical queries
2. How Parquet compares to CSV in file size and read latency
3. How column projection avoids reading irrelevant data entirely
4. Where Parquet fits in the broader data ecosystem

In [103]:
import io
import os
import time

import pyarrow as pa
import pyarrow.csv as pa_csv
import pyarrow.parquet as pq

---

## 1. Build a sample dataset

> **Spec:** [Overview](https://parquet.apache.org/docs/overview/)

We create a table with 1 million rows and four columns of different types.
This is large enough for the size and speed differences to be meaningful.

In [ ]:
N = 1_000_000
categories = ["alpha", "beta", "gamma", "delta", "epsilon"]

table = pa.table({
    "id":       pa.array(range(N), type=pa.int64()),
    "value":    pa.array([float(i) * 0.1 for i in range(N)], type=pa.float64()),
    "category": pa.array([categories[i % len(categories)] for i in range(N)], type=pa.string()),
    "flag":     pa.array([i % 2 == 0 for i in range(N)], type=pa.bool_()),
})

print(f"Rows: {table.num_rows:,}")
print(f"Columns: {table.num_columns}")
print(f"Schema: {table.schema}")

Rows: 20,000,000
Columns: 4
Schema: id: int64
value: double
category: string
flag: bool


---

## 2. Write as CSV and Parquet, compare file size

> **Spec:** [Motivation: compression](https://parquet.apache.org/docs/overview/motivation/)

Parquet applies per-column encoding and compression by default.  
CSV stores every value as a plain text character, with no type awareness.

In [105]:
csv_path = "/tmp/sample.csv"
parquet_path = "/tmp/sample.parquet"

os.remove(csv_path) if os.path.exists(csv_path) else None
os.remove(parquet_path) if os.path.exists(parquet_path) else None

# Write CSV
csv_buf = io.BytesIO()
pa_csv.write_csv(table, csv_buf)
with open(csv_path, "wb") as f:
    f.write(csv_buf.getvalue())

# Write Parquet
pq.write_table(table, parquet_path, compression='zstd', row_group_size=N / 50)

csv_size = os.path.getsize(csv_path)
parquet_size = os.path.getsize(parquet_path)
ratio = csv_size / parquet_size

print(f"CSV size:     {csv_size / 1024 / 1024:.2f} MB")
print(f"Parquet size: {parquet_size / 1024 / 1024:.2f} MB")
print(f"Ratio:        Parquet is {ratio:.1f}x smaller than CSV")

CSV size:     662.20 MB
Parquet size: 66.85 MB
Ratio:        Parquet is 9.9x smaller than CSV


---

## 3. Read latency: full table

> **Spec:** [Motivation](https://parquet.apache.org/docs/overview/motivation/)

Even reading the entire file, Parquet is faster because the on-disk data is
more compact (less I/O) and each column is decoded with type-specific routines.

In [ ]:
RUNS = 3

def bench(fn, label):
    times = []
    cpu_times = []
    for _ in range(RUNS):
        t0 = time.perf_counter()
        cpu_t0 = time.process_time()
        fn()
        times.append(time.perf_counter() - t0)
        cpu_times.append(time.process_time() - cpu_t0)
    avg = sum(times) / RUNS
    cpu_avg = sum(cpu_times) / RUNS
    print(f"{label}: {avg * 1000:.1f} ms, {cpu_avg * 1000:.1f} CPU ms, (avg of {RUNS} runs)")
    return avg

t_csv = bench(lambda: pa_csv.read_csv(csv_path), "Read CSV")
t_pq  = bench(lambda: pq.read_table(parquet_path), "Read Parquet")
print(f"Speedup: Parquet is  {t_csv / t_pq:.1f}x faster to read than CSV")

Read CSV: 189.0 ms, 2713.9 CPU ms, (avg of 5 runs)
Read Parquet: 118.7 ms, 1169.1 CPU ms, (avg of 5 runs)
Speedup: Parquet is  1.6x faster to read than CSV


---

## 4. Column projection: read only what you need

> **Spec:** [Concepts: Unit of parallelization](https://parquet.apache.org/docs/concepts/)

Parquet stores each column chunk contiguously on disk.  
Requesting a subset of columns means the other column chunks are **never read from disk**.

CSV has no such capability, every byte must be scanned to find field boundaries.

In [109]:
t_csv_proj = bench(
    lambda: pa_csv.read_csv(csv_path),  # CSV must still read everything
    "CSV   (all columns, no choice)"
)

t_pq_proj = bench(
    lambda: pq.read_table(parquet_path, columns=["value"]),  # Only 1 of 4 columns
    "Parquet (1 column projected)   "
)

print(f"Speedup from projection: {t_csv_proj / t_pq_proj:.1f}x")

CSV   (all columns, no choice): 174.0 ms, 2500.7 CPU ms, (avg of 5 runs)
Parquet (1 column projected)   : 61.3 ms, 304.2 CPU ms, (avg of 5 runs)
Speedup from projection: 2.8x


---

## Summary

| Concept | Key point |
|---------|----------|
| Columnar storage | Values of the same column are stored together, it's ideal for aggregations |
| Compression | Type-aware encoding (dictionary, delta, RLE) dramatically reduces size |
| Column projection | Only columns actually requested are read from disk |
| Row group size | Smaller groups -> finer predicate skipping. Larger groups -> better compression & throughput |
| Ecosystem | `pyarrow.parquet` wraps the Arrow C++ Parquet library|

**Next ⏭️** [File format](02_file_format.ipynb)

---

## 5. Extra: Row group size, the tuning knob for predicate pushdown

> **Spec:** [Concepts](https://parquet.apache.org/docs/concepts/)

Each row group stores its own min/max statistics per column.
When a filter is applied, the reader evaluates those statistics and skips entire
row groups that cannot contain matching rows.

**The trade-off:**
- **Fewer, larger row groups** → better compression and sequential I/O throughput, but coarser skipping.
- **More, smaller row groups** → better parallelization, finer-grained predicate skipping, but higher metadata overhead and more random I/O.

We write the same table four times with different `row_group_size` values and then
benchmark a highly selective filter to observe the effect.

In [ ]:
# Write the same table with four different row group sizes
rg_configs = {
    "rg_10k":  10_000,
    "rg_100k": 100_000,
    "rg_500k": 500_000,
    "rg_2M":   2_000_000,
}

paths_rg = {}
for name, rg_size in rg_configs.items():
    p = f"/tmp/sample_{name}.parquet"
    pq.write_table(table, p, compression="zstd", row_group_size=rg_size)
    meta = pq.read_metadata(p)
    paths_rg[name] = (p, rg_size, meta.num_row_groups)
    print(f"  {name}: row_group_size={rg_size:>9,}  -> {meta.num_row_groups:>4} row groups  "
          f"  file size: {os.path.getsize(p) / 1024 / 1024:.1f} MB")

print()

# Benchmark a selective filter: only the last 1 % of rows match
threshold = int(N * 0.99)
filter_expr = [("id", ">=", threshold)]

print(f"Filter: id >= {threshold:,}  ({N - threshold:,} matching rows out of {N:,})\n")
print(f"{'Config':<12} {'RG size':>10} {'# RGs':>6} {'Read time (ms)':>16} {'RGs skipped':>12}")
print("-" * 62)

results_rg = {}
for name, (p, rg_size, num_rgs) in paths_rg.items():
    times = []
    for _ in range(RUNS):
        t0 = time.perf_counter()
        pq.read_table(p, filters=filter_expr)
        times.append(time.perf_counter() - t0)
    avg_ms = sum(times) / RUNS * 1000

    # Count how many RGs are actually skipped by inspecting statistics
    pf = pq.ParquetFile(p)
    meta = pf.metadata
    skipped = 0
    for rg_idx in range(meta.num_row_groups):
        rg = meta.row_group(rg_idx)
        for col_idx in range(meta.num_columns):
            col = rg.column(col_idx)
            if col.path_in_schema == "id":
                if col.statistics.max < threshold:
                    skipped += 1
                break

    results_rg[name] = avg_ms
    print(f"{name:<12} {rg_size:>10,} {num_rgs:>6}   {avg_ms:>12.1f} ms   {skipped:>5}/{num_rgs} skipped ({skipped / num_rgs * 100:.1f}%)")

best = min(results_rg, key=results_rg.get)
worst = max(results_rg, key=results_rg.get)
print(f"\nBest config: {best} ({results_rg[best]:.1f} ms:  "
      f"{results_rg[worst] / results_rg[best]:.1f}x faster than {worst} ({results_rg[worst]:.1f} ms)")

  rg_10k: row_group_size=   10,000  -> 2000 row groups    file size: 383.1 MB
  rg_100k: row_group_size=  100,000  ->  200 row groups    file size: 392.4 MB
  rg_500k: row_group_size=  500,000  ->   40 row groups    file size: 335.7 MB
  rg_2M: row_group_size=2,000,000  ->   10 row groups    file size: 320.2 MB

Filter: id >= 19,800,000  (200,000 matching rows out of 20,000,000)

Config          RG size  # RGs   Read time (ms)  RGs skipped
--------------------------------------------------------------
rg_10k           10,000   2000           33.6 ms    1980/2000 skipped
rg_100k         100,000    200           14.5 ms     198/200 skipped
rg_500k         500,000     40           18.1 ms      39/40 skipped
rg_2M         2,000,000     10           56.1 ms       9/10 skipped

Best config: rg_100k (14.5 ms:  3.9x faster than rg_2M (56.1 ms)
